# Домашняя работа по теме "Машинное обучение ранжированию"

В этом ДЗ мы:
- научимся работать со стандартным датасетом для машинного обучения ранжированию [MSLR](https://www.microsoft.com/en-us/research/project/mslr/)
- попробуем применить на практике все то, чему мы научились на семинаре

## Как будет происходить сдача ДЗ

Вам надо:
- форкнуть эту репу
- создать бранч в котором вы дальше будете работать
- реализовать класс Model в этом ноутбуке
- убедиться, что ваша реализация выбивает NDCG@10 выше бейзлайна (см. ниже)
- запушить ваш бранч и поставить Pull Request
- в комментарии написать какой скор вы выбили

В таком случае мы (организаторы):

- счекаутим вашу бранчу
- проверим что ваша реализация действительно выбивает заявленный скор

Предполагается, что и вы, и мы работаем в виртаульном окружении как в семинаре про машинное обучение ранжированию: seminars/7-learning-to-rank/requirements.txt(подробнее про работу с виртуальными окружениями README в корне этой репы).

Оценка:
- За выбитый скор больше **0.507** назначаем **5** баллов, за скор больше (или равно) **0.510** назначаем максимальный балл -- 10 баллов
- Тот из участников кто выбъет самый высокий скор получит еще +10 баллов

При сдаче кода важно помнить о том, что:
- В коде не должно быть захардкоженных с потолка взятых гиперпараметров (таких как число деревьев, learning rate и т.п.) -- обязательно должен быть представлен код который их подбирает!
- Решение должно быть стабильно от запуска к запуску (на CPU) т.е. все seed'ы для генераторов случайных чисел должны быть фиксированы
- Мы (организаторы) будем запускать код на CPU поэтому, даже если вы использовали для подбора параметров GPU, финальный скор надо репортить на CPU

## Пререквизиты

Импортируем все что нам понадобится для дальнейшей работы:

In [2]:
import pathlib
from timeit import default_timer as timer

import numpy as np
import pandas as pd

from catboost import Pool,CatBoost,datasets, utils

## Датасет MSLR (Microsoft Learning to Rank)

Загрузим датасет MSLR.

Полный датасет можно скачать с официального сайта: https://www.microsoft.com/en-us/research/project/mslr/

Строго говоря, он состоит их 2х частей:

- основной датасет MSLR-WEB30K -- он содержит более 30 тыс. запросов
- "маленький" датасет MSLR-WEB10K, который содержит только 10 тыс. запросов и является случайным сэмплом датасета MSLR-WEB30K

в этом ДЗ мы будем работать с MSLR-WEB10K, т.к. полная версия датасета может просто не поместиться у нас в RAM (и, тем более, в память видеокарты если мы учимся на GPU)

Будем считать, что мы самостоятельно скачали датасет MSLR-WEB10K с официального сайта, поместили его в папку КОРЕНЬ-ЭТОЙ-РЕПЫ/data/mslr-web10k и раззиповали.

В результате у нас должна получиться следующая структура папок:

In [3]:
# ls ../../data
# mslr-web10k

In [4]:
# # ls -lh ../../data/mslr-web10k/
# итого 1,2G
# drwxr-xr-x 2 andrei andrei 4,0K апр 28  2010 Fold1
# drwxr-xr-x 2 andrei andrei 4,0K апр 28  2010 Fold2
# drwxr-xr-x 2 andrei andrei 4,0K апр 28  2010 Fold3
# drwxr-xr-x 2 andrei andrei 4,0K апр 28  2010 Fold4
# drwxr-xr-x 2 andrei andrei 4,0K апр 28  2010 Fold5
# -rw-r--r-- 1 andrei andrei 1,2G июл  7  2016 MSLR-WEB10K.zip

Заметим, что датасет довольно большой, в распакованном виде он весит 7.7 GB.

Датасет состоит из нескольких фолдов, которые по сути представляют из себя разные разбиения одних и тех же данных на обучающее, валидационное и тестовые множеста.

Дальше мы будем использовать только первый фолд: Fold1.

Заглянем внутрь:

In [5]:
#!ls -lh ../../data/mslr-web10k/Fold1
# итого 1,3G
# -rw-r--r-- 1 andrei andrei 267M апр 30  2010 test.txt
# -rw-r--r-- 1 andrei andrei 800M апр 30  2010 train.txt
# -rw-r--r-- 1 andrei andrei 261M апр 30  2010 vali.txt

Видим, что у нас 3 файла с говорящими названиями, соответсвующими сплитам нашего датасета.

Посмотрим на содержимое одного из файлов:

In [6]:
#!head -n 1 ../../data/mslr-web10k/Fold1/train.txt
#2 qid:1 1:3 2:3 3:0 4:0 5:3 6:1 7:1 8:0 9:0 10:1 11:156 12:4 13:0 14:7 15:167 16:6.931275 17:22.076928 18:19.673353 19:22.255383 20:6.926551 21:3 22:3 23:0 24:0 25:6 26:1 27:1 28:0 29:0 30:2 31:1 32:1 33:0 34:0 35:2 36:1 37:1 38:0 39:0 40:2 41:0 42:0 43:0 44:0 45:0 46:0.019231 47:0.75000 48:0 49:0 50:0.035928 51:0.00641 52:0.25000 53:0 54:0 55:0.011976 56:0.00641 57:0.25000 58:0 59:0 60:0.011976 61:0.00641 62:0.25000 63:0 64:0 65:0.011976 66:0 67:0 68:0 69:0 70:0 71:6.931275 72:22.076928 73:0 74:0 75:13.853103 76:1.152128 77:5.99246 78:0 79:0 80:2.297197 81:3.078917 82:8.517343 83:0 84:0 85:6.156595 86:2.310425 87:7.358976 88:0 89:0 90:4.617701 91:0.694726 92:1.084169 93:0 94:0 95:2.78795 96:1 97:1 98:0 99:0 100:1 101:1 102:1 103:0 104:0 105:1 106:12.941469 107:20.59276 108:0 109:0 110:16.766961 111:-18.567793 112:-7.760072 113:-20.838749 114:-25.436074 115:-14.518523 116:-21.710022 117:-21.339609 118:-24.497864 119:-27.690319 120:-20.203779 121:-15.449379 122:-4.474452 123:-23.634899 124:-28.119826 125:-13.581932 126:3 127:62 128:11089534 129:2 130:116 131:64034 132:13 133:3 134:0 135:0 136:0 

Видим, что данные лежат в уже знакомом нам по семинару формате:

- В первой колонке лежит таргет (оценка асессора), по 5-балльной шкале релевантности: от 0 до 4 (включительно)
- Во второй колонке лежит ID запроса, по которому можно сгруппировать все оценки документов в рамках одного и того же запроса
- Дальше идет вектор из 128 фичей (таких как значения BM25 и т.п.), их точная природа нам сейчас на важна

В файле qid и все-фичи кодируются в формате КЛЮЧ:ЗНАЧЕНИЕ, напр. 130:116 -- тут 130 это номер фичи, а 116 -- ее значение.

Такой формат в мире машинного обучения часто называют svm light формат (в честь когда-то популярной библиотеки SVM-Light)

Напишем немного вспомогательного кода для загрузки этого датасета:

In [7]:
def generate_column_names(num_features):
    """Generates column names for LETOR-like datasets"""
    columns = ['label', 'qid']
    for i in range(num_features):
        column = f"feature_{i+1}"
        columns.append(column)
    return columns
    
def load_svmlight_file(input_file, max_num_lines=0):
    """Loads dataset split in SVM-Light format"""
    def _parse_field(field):
        parts = field.split(':')
        if len(parts) != 2:
            raise Exception(f"invalid number of parts in field {field}")
        return parts

    num_features = 136
    exp_num_fields = num_features + 2
    num_lines = 0
    X = []
    with open(input_file, 'rt') as f:
        for line in f:
            try:
                num_lines += 1
                                  
                # Parse into fields
                fields = line.rstrip().split(' ')
                num_fields = len(fields)
                if num_fields != exp_num_fields:
                    raise Exception(f"invalid number of fields {num_fields}")
    
                # Parse every field
                x = np.zeros(exp_num_fields, dtype=np.float32)
                label = int(fields[0])
                x[0] = label
                _, qid_str = _parse_field(fields[1])
                qid = int(qid_str)
                x[1] = qid
                for i, field in enumerate(fields[2:]):
                    _, feature_str = _parse_field(field)
                    x[i+2] = float(feature_str)
    
                # Add new object
                X.append(x)
                if num_lines % 50000 == 0:
                    print(f"Loaded {num_lines} lines...")
                if max_num_lines > 0 and num_lines == max_num_lines:
                    print(f"WARNING: stop loading, line limit reached: max_num_lines = {max_num_lines} input_file = {input_file}")
                    break
            except Exception as e:
                raise Exception(f"error at line {num_lines} in {input_file}") from e
    
    # To pandas
    df = pd.DataFrame(X, columns=generate_column_names(num_features))
    print(f"Loaded SVM-Light file {input_file}")
    return df

И теперь загрузим датасет:

In [8]:
# возможно здесь придётся привести папку mslr... к верхнему регистру
# в windows пути к файлам case-insensitive
fold_dir = pathlib.Path("../../data/mslr-web10k/Fold1")

df_train = load_svmlight_file(fold_dir.joinpath("train.txt"))
df_valid = load_svmlight_file(fold_dir.joinpath("vali.txt"))
df_test = load_svmlight_file(fold_dir.joinpath("test.txt"))
print(f"Dataset loaded from fold_dir {fold_dir}")

Loaded 50000 lines...
Loaded 100000 lines...
Loaded 150000 lines...
Loaded 200000 lines...
Loaded 250000 lines...
Loaded 300000 lines...
Loaded 350000 lines...
Loaded 400000 lines...
Loaded 450000 lines...
Loaded 500000 lines...
Loaded 550000 lines...
Loaded 600000 lines...
Loaded 650000 lines...
Loaded 700000 lines...
Loaded SVM-Light file ..\..\data\mslr-web10k\Fold1\train.txt
Loaded 50000 lines...
Loaded 100000 lines...
Loaded 150000 lines...
Loaded 200000 lines...
Loaded SVM-Light file ..\..\data\mslr-web10k\Fold1\vali.txt
Loaded 50000 lines...
Loaded 100000 lines...
Loaded 150000 lines...
Loaded 200000 lines...
Loaded SVM-Light file ..\..\data\mslr-web10k\Fold1\test.txt
Dataset loaded from fold_dir ..\..\data\mslr-web10k\Fold1


## Обучаем модель

Теперь можно приступить непосредственно к обучению модели. 

Объявим класс модели, который надо будем заимлементить в этом ДЗ:

In [9]:

class Model:
    # params = None if model will be loaded
    def __init__(self,params=None):
        self.model = None 
        self.params = params

    def fit(self, df_train,df_val = None ):
        X_train,y_train,q_train = self.to_catboost_dataset(df_train)
        pool_train = Pool(data=X_train, label=y_train, group_id=q_train)

        X_val,y_val,q_val = self.to_catboost_dataset(df_val)
        pool_val = Pool(data=X_val,label=y_val,group_id=q_val)

        

        self.model = CatBoost(self.params)

    
        self.model.fit(pool_train,eval_set=pool_val)
            


    def predict(self, df_test):
        X_test,y_test,q_test = self.to_catboost_dataset(df_test)
        pool_test = Pool(data=X_test, label=y_test, group_id=q_test)
        return self.model.predict(pool_test)

    # вызывать после выполнения fit
    def save_model(self):
        self.model.save_model("ranking_model",format="cbm")
        
    def load_model(self):
        self.model = CatBoost()
        self.model.load_model("ranking_model",format="cbm")
        
    def score(self,df_test):
        eval_metric = 'NDCG:top=10;type=Exp'

        X_test,y_test,q_test = self.to_catboost_dataset(df_test)
        y_test = df_test['label'].to_numpy()
        q_test = df_test['qid'].to_numpy().astype('uint32')

        y_predict = self.predict(df_test)

        score = utils.eval_metric(y_test, y_predict, eval_metric, group_id=q_test)
        return score[0]



    def to_catboost_dataset(self, df):
        y = df['label'].to_numpy()                       # Label: [0-4]
        q = df['qid'].to_numpy().astype('uint32')        # Query Id
        X = df.drop(columns=['label', 'qid']).to_numpy() # 136 features
        return (X, y, q)


Обучение модели на лучших параметрах:

In [23]:
# ячейка для обучения с лучшими параметрами 
# eё также не надо запускать, если вы хотите просто проверить скор модели 
import json 

with open("best_hyp.json","r") as param_file:
    best_params = json.load(param_file)


print(best_params)
model = Model(best_params)



# Fit
start = timer()
model.fit(df_train, df_valid)
elapsed = timer() - start
print(f"Model fit: elapsed = {elapsed:.3f}")

# сохраняем обученную модель
model.save_model()

print("Скор модели на тестовом датасете")
print(model.score(df_test))

{'loss_function': 'YetiRank:mode=Classic;num_neighbours=2', 'iterations': 2000, 'depth': 9, 'learning_rate': 0.05, 'min_data_in_leaf': 1, 'use_best_model': True, 'eval_metric': 'NDCG:top=10;type=Exp', 'early_stopping_rounds': 100, 'random_seed': 22, 'verbose': 10, 'l2_leaf_reg': 0.03}
0:	test: 0.3400908	best: 0.3400908 (0)	total: 461ms	remaining: 15m 22s
10:	test: 0.4356727	best: 0.4356727 (10)	total: 4.8s	remaining: 14m 28s
20:	test: 0.4536279	best: 0.4536279 (20)	total: 9.23s	remaining: 14m 29s
30:	test: 0.4639935	best: 0.4639935 (30)	total: 13.6s	remaining: 14m 23s
40:	test: 0.4732485	best: 0.4732485 (40)	total: 18s	remaining: 14m 21s
50:	test: 0.4779903	best: 0.4779903 (50)	total: 22.5s	remaining: 14m 18s
60:	test: 0.4841248	best: 0.4841248 (60)	total: 27.7s	remaining: 14m 39s
70:	test: 0.4883553	best: 0.4883553 (70)	total: 32.8s	remaining: 14m 52s
80:	test: 0.4914837	best: 0.4914837 (80)	total: 38s	remaining: 15m
90:	test: 0.4940050	best: 0.4940050 (90)	total: 43.1s	remaining: 15m

In [39]:
# ячейка для проверки скора 
# предварительно надо выполнить ячейки связанные с импортированием библиотек и 
# загрузкой + обработкой датасета

with open("best_hyp.json","r") as param_file:
    best_params = json.load(param_file)

model = Model(params = None)


model.load_model(best_params)

print(model.score(df_test))





TypeError: Model.__init__() missing 1 required positional argument: 'params'

Ожидаем, что ваша модель покажет результаты выше бейзлайна!